In [ ]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [ ]:
conf = Config()

conf.dataset = 'usa_large_cellyc'
conf.post_value_updates()

In [ ]:
model = TrajCL()

In [ ]:
model

In [ ]:
# from ah_databricks_data_loader import DatabricksDataLoader

# # Instantiate the DatabricksDataLoader.
# ddl = DatabricksDataLoader()

# # Load data.
# test_df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_traj_data_v18")

In [ ]:
# test_df.count()

In [ ]:
# test_df.show(5)

In [ ]:
# test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/nyc/test_hyp1/nyc_data_v18")

In [ ]:
# test_df.count()

In [ ]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/nyc/test_hyp1/nyc_data_v18/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

In [ ]:
df_list[0].head()

In [ ]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [ ]:
# userids = test_df['userid'].unique()

In [ ]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/la_nyc_emb_add/usa_large_cell_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

In [ ]:
from utils.traj import *
import pickle
from utils.cellspace import *

from torch.nn.utils.rnn import pad_sequence
embs_parent = pickle.load(open(Config.dataset_embs_file_parent, 'rb')).to('cpu').detach() # tensor
embs_child = pickle.load(open(Config.dataset_embs_file_child, 'rb')).to('cpu').detach() # tensor
cellspace_parent = pickle.load(open(Config.dataset_cell_file_parent, 'rb'))
cellspace_child = pickle.load(open(Config.dataset_cell_file_child, 'rb'))
hier_cellspace = HirearchicalCellSpace(cellspace_parent, cellspace_child)

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell_parent, traj_cell_child, traj_p = zip(*[merc2cell2(t, hier_cellspace) for t in traj])
    # print(traj_cell)
    traj_emb_p = [torch.tensor(generate_spatial_features(t, hier_cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell_parent = [embs_parent[list(t)] for t in traj_cell_parent]
    traj_emb_cell_child = [embs_child[list(t)] for t in traj_cell_child]
    traj_emb_cell = [a + b for a, b in zip(traj_emb_cell_parent, traj_emb_cell_child)]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_p)), dtype = torch.long, device = device)
    time_indices = pad_sequence([torch.tensor(t, dtype=torch.long) for t in time_indices], batch_first=False, padding_value=-1).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, time_indices)
    return traj_embs

def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size] 
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
df_list[0]['time_index_list'].values[0]

In [ ]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

In [ ]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['time_index_list'].values
    return traj, time_indices

In [ ]:
def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [ ]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    count = 0
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                continue
            else:
                # print(userid, employer, partition)
                # print(len(part_data))
                traj, time_indices = get_traj_and_time_data(part_data)
                # print(traj)
                embs = infer(traj, time_indices).detach().cpu().numpy()
                db_scan = apply_dbscan(embs, target_min_similarity=0.85)
                labels = db_scan.labels_                      # shape: (n_samples,)
                cluster_ids = [c for c in np.unique(labels) if c != -1]
                if len(cluster_ids)>0:
                    pred_label = 1
                else:
                    pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    return count
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label
count=0
for userid in tqdm(unique_userids):
    count+=get_gt_and_pred_label(userid)


        


  2%|▏         | 423/25831 [00:41<37:28, 11.30it/s]

In [ ]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/la_nyc_emb_add/nyc_test_hyp1_output_dict.pkl", 'wb'))

In [ ]:
output_dict

In [ ]:
count_dict = {}
for k,v in output_dict.items():
    if v in count_dict:
        count_dict[v]+=1
    else:
        count_dict[v]=1
count_dict

In [ ]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part_fil = df_part_fil[df_part_fil['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part_fil

In [ ]:
df_part = get_data_userid_employer_partition(34299, 'u s postal service', 0)
df_part.sort_values('traj_date')